In [1]:
import pandas as pd
import numpy as np

In [2]:
# load the biobank and the disease attribute
biobank = pd.read_csv('data/sample_catalogue_rd_connect.csv')
disease_rd = pd.read_csv('data/rdconnect_Disease_2017-10-04_02-59-41.csv')

In [3]:
biobank.head()

,Disease,MaterialType,MaterialTypeDetailed,AnatomicalSite,Sex,DiagnosisType,Genotype,AgeAtSampling,Affected,Family,RelatedSamples,Registry,BiobankID,RegistryID
0,urn:miriam:ncit:C46951,SERUM,http://purl.obolibrary.org/obo/FMA_63083,http://purl.obolibrary.org/obo/UBERON_0000178,http://purl.obolibrary.org/obo/PATO_0000384,CLINICAL,FALSE,15.0,TRUE,UNK,NAV,UNK,87919,NaN
1,urn:miriam:ncit:C46951,SERUM,http://purl.obolibrary.org/obo/FMA_63083,http://purl.obolibrary.org/obo/UBERON_0000178,http://purl.obolibrary.org/obo/PATO_0000384,CLINICAL,FALSE,20.0,TRUE,UNK,NAV,UNK,87919,NaN
2,urn:miriam:orphanet:Orphanet_101075,OTHER,http://purl.obolibrary.org/obo/FMA_63877,http://purl.obolibrary.org/obo/UBERON_0002097,http://purl.obolibrary.org/obo/PATO_0000384,MOLECULAR,TRUE,26.0,TRUE,UNK,UNK,UNK,87919,NaN
3,urn:miriam:orphanet:Orphanet_101075,OTHER,http://purl.obolibrary.org/obo/FMA_63877,http://purl.obolibrary.org/obo/UBERON_0002097,http://purl.obolibrary.org/obo/PATO_0000384,MOLECULAR,TRUE,26.0,TRUE,UNK,UNK,UNK,87919,NaN
4,urn:miriam:orphanet:Orphanet_101081,SERUM,http://purl.obolibrary.org/obo/FMA_63083,http://purl.obolibrary.org/obo/UBERON_0000178,http://purl.obolibrary.org/obo/PATO_0000384,MOLECULAR,TRUE,19.0,TRUE,UNK,UNK,UNK,87919,NaN


In [4]:
disease_rd.head()

,IRI,PreferredTerm,Description,Code
0,http://www.orpha.net/ORDO/Orphanet_1440,Ring chromosome 14,Ring chromosome 14,1440
1,http://identifiers.org/ncit/C49651,Healthy Subject,An individual who is or becomes a participant ...,C49651
2,urn:miriam:orphanet:Orphanet_802,Multiple sclerose,NaN,802
3,urn:miriam:orphanet:Orphanet_424925,Qualitative or quantitative defects of Torsin-...,NaN,424925
4,urn:miriam:orphanet:Orphanet_424933,Rare epithelial tumor of liver and intrahepati...,NaN,424933


In [5]:
# Name and Code for the disease
print(biobank.Disease[0])
print(disease_rd.Code[disease_rd.IRI==biobank.Disease[0]])

urn:miriam:ncit:C46951
2783    46951
Name: Code, dtype: object


In [3]:
biobank.groupby('Disease').count().shape

(91, 13)

In [4]:
# load the mapping between diseases and symptons' HPO id
names = pd.read_csv('data/ALL_SOURCES_ALL_FREQUENCIES_diseases_to_genes_to_phenotypes.txt',
                    sep=('<tab>'), engine='python')
names_list = [str(item) for item in names.columns]
names_list[0] = names_list[0].split(': ')[1]
mappings_disease_sympton = pd.read_csv('data/ALL_SOURCES_ALL_FREQUENCIES_diseases_to_genes_to_phenotypes.txt',
                                        sep=('\t'), names=names_list, skiprows=1)
print(mappings_disease_sympton.head())

     diseaseId gene-symbol  gene-id(entrez)      HPO-ID  \
0  OMIM:600142       HTRA1             5654  HP:0000007   
1  OMIM:600142       HTRA1             5654  HP:0001251   
2  OMIM:600142       HTRA1             5654  HP:0002448   
3  OMIM:600142       HTRA1             5654  HP:0003419   
4  OMIM:600142       HTRA1             5654  HP:0007204   

                        HPO-term-name  
0     Autosomal recessive inheritance  
1                              Ataxia  
2          Progressive encephalopathy  
3                       Low back pain  
4  Diffuse white matter abnormalities  


In [58]:
mappings_disease_sympton.groupby('diseaseId').count().diseaseCode.max()

1800

In [5]:
# extract disease Code from diseaseID
mappings_disease_sympton['diseaseCode'] = mappings_disease_sympton.diseaseId
def extract_code(diseaseID):
    diseaseCode = str(diseaseID).split(':')[1]
    return diseaseCode
mappings_disease_sympton.diseaseCode = mappings_disease_sympton.diseaseCode.apply(extract_code)

In [6]:
# match disease in biobank with diseaseCode from diease_rd database
biobank['diseaseCode'] = biobank.Disease
def mapping_disease_biobank_diseaserd(diseaseName):
    return disease_rd.Code[disease_rd.IRI == diseaseName].values[0]
biobank['diseaseCode'] = biobank.diseaseCode.apply(mapping_disease_biobank_diseaserd)
print(biobank.diseaseCode.head())

0     46951
1     46951
2    101075
3    101075
4    101081
Name: diseaseCode, dtype: object


In [7]:
biobank['Disease'][19]

'urn:miriam:orphanet:Orphanet_103'

In [39]:
mappings_disease_sympton[['diseaseCode','HPO-ID']].head()

,diseaseCode,HPO-ID
0,600142,HP:0000007
1,600142,HP:0001251
2,600142,HP:0002448
3,600142,HP:0003419
4,600142,HP:0007204


In [96]:
mappings_disease_sympton['HPO-ID'][disease_bank.diseaseCode[5]==
                                   mappings_disease_sympton.diseaseCode].values


array(['HP:0200125', 'HP:0000622', 'HP:0000603', 'HP:0003198',
       'HP:0007763', 'HP:0007924', 'HP:0000576', 'HP:0004309',
       'HP:0001251', 'HP:0012841', 'HP:0009830', 'HP:0002174',
       'HP:0000648', 'HP:0200125', 'HP:0000622', 'HP:0000603',
       'HP:0003198', 'HP:0007763', 'HP:0007924', 'HP:0000576',
       'HP:0004309', 'HP:0001251', 'HP:0012841', 'HP:0009830',
       'HP:0002174', 'HP:0000648', 'HP:0200125', 'HP:0000622',
       'HP:0000603', 'HP:0003198', 'HP:0007763', 'HP:0007924',
       'HP:0000576', 'HP:0004309', 'HP:0001251', 'HP:0012841',
       'HP:0009830', 'HP:0002174', 'HP:0000648', 'HP:0200125',
       'HP:0000622', 'HP:0000603', 'HP:0003198', 'HP:0007763',
       'HP:0007924', 'HP:0000576', 'HP:0004309', 'HP:0001251',
       'HP:0012841', 'HP:0009830', 'HP:0002174', 'HP:0000648',
       'HP:0200125', 'HP:0000622', 'HP:0000603', 'HP:0003198',
       'HP:0007763', 'HP:0007924', 'HP:0000576', 'HP:0004309',
       'HP:0001251', 'HP:0012841', 'HP:0009830', 'HP:00

In [8]:
# find the mapped symptonsHPO_ID for a diseaseCode from biobank
disease_with_symptons = {}
# all the disease in the biobank with their names and codes
disease_bank = biobank[['Disease','diseaseCode']].groupby('Disease').max() 
# disease_bank.head() # show the disease_bank
disease_names = disease_bank.index.values
disease_codes = disease_bank.diseaseCode.values
print(disease_names[:5], disease_codes[:5])
for i in range(len(disease_names)):
    if disease_names[i] not in disease_with_symptons:
        disease_with_symptons[disease_names[i]] = \
        mappings_disease_sympton['HPO-ID'][disease_codes[i]==
                                           mappings_disease_sympton.diseaseCode].values
        


['urn:miriam:ncit:C46951' 'urn:miriam:orphanet:Orphanet_101075'
 'urn:miriam:orphanet:Orphanet_101081'
 'urn:miriam:orphanet:Orphanet_101082' 'urn:miriam:orphanet:Orphanet_103'] ['46951' '101075' '101081' '101082' '103']


In [9]:
# select the diseases with symptons and corresponding HPO-IDs
disease_has_symptons = {}
for key,value in disease_with_symptons.items():
    if len(value) is not 0:
        disease_has_symptons[key] = value
        print(value)

['HP:0007703' 'HP:0000613' 'HP:0000512' 'HP:0007939' 'HP:0000505'
 'HP:0001131' 'HP:0007703' 'HP:0000613' 'HP:0000512' 'HP:0007939'
 'HP:0000505' 'HP:0001131']
['HP:0000505' 'HP:0001251' 'HP:0000486' 'HP:0000551' 'HP:0001249'
 'HP:0001257' 'HP:0000648' 'HP:0000639' 'HP:0000505' 'HP:0001251'
 'HP:0000486' 'HP:0000551' 'HP:0001249' 'HP:0001257' 'HP:0000648'
 'HP:0000639']
['HP:0000518' 'HP:0002486']
['HP:0002486' 'HP:0000818' 'HP:0000518' 'HP:0004299' 'HP:0100335'
 'HP:0010864' 'HP:0000486' 'HP:0011705' 'HP:0000028' 'HP:0011362'
 'HP:0001252' 'HP:0010935' 'HP:0003202' 'HP:0010628' 'HP:0000238'
 'HP:0000298' 'HP:0002098' 'HP:0002827' 'HP:0001276' 'HP:0003457'
 'HP:0006887' 'HP:0030680' 'HP:0000029']
['HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316'
 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316'
 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316'
 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316' 'HP:0008316'
 'HP:0008316' 'HP:0008316' 'HP

In [10]:
from owlready2 import *
from rdflib import *
from rdflib.plugins.sparql import prepareQuery
from rdflib import URIRef
owlURL = "http://purl.obolibrary.org/obo/hp.owl"
# load ontology
onto = get_ontology(owlURL)
onto.load()

# load graph
my_world = World()
my_world.get_ontology(owlURL).load()
graph = my_world.as_rdflib_graph()

In [11]:
term1 = onto.search_one(iri="*HP_0100148")
term2 = onto.search_one(iri="*HP_0000011")
print(term1,term1.iri)
print(term2, term2.iri)

obo.HP_0100148 http://purl.obolibrary.org/obo/HP_0100148
obo.HP_0000011 http://purl.obolibrary.org/obo/HP_0000011


In [12]:
# calculate the distance between two arbitary nodes
q = prepareQuery("""
                SELECT (count(?p) as ?distance)
                WHERE{
                    ?object1 <http://www.w3.org/2000/01/rdf-schema#subClassOf>* ?p .
                    ?object2 <http://www.w3.org/2000/01/rdf-schema#subClassOf>* ?p .
                    }
                """)
# for term in [term1, term2]:
#     object_uri = URIRef(term.iri)
#     r = list(graph.query(q,initBindings={'object':object_uri}))
#     print(r)

object1_iri = URIRef(term1.iri)
object2_iri = URIRef(term2.iri)
r = list(graph.query(q,initBindings={'object1':object1_iri,'object2':object2_iri}))    

# r = list(graph.query("""SELECT ?r ?p 
# WHERE {
#   <term1> ?r ?p ,
#   initNs={'term1':term1.iri}.
# }"""))

In [24]:
q = prepareQuery("""
                SELECT (count(?p) as ?distance)
                WHERE{
                    ?object1 <http://www.w3.org/2000/01/rdf-schema#subClassOf>* ?p .
                    ?object2 <http://www.w3.org/2000/01/rdf-schema#subClassOf>* ?p .
                    }
                """)
def pairwise_distance_two_diseases(node1, node2, graph, query):
    '''
    Count the in-between nodes,
    Input: node1,node2: URI, str
           graph: rdflib graph
           query: prepareQuery
    Output: distance: int
    '''
    distance = list(graph.query(query,initBindings={'object1':node1,
                                                'object2':node2})) 
    return distance

In [16]:
disease_number = len(disease_has_symptons.keys())
disease_between_diseases = np.zeros((disease_number, disease_number),np.int8)
i = 0
disease1 = []
disease2 = []
for key, value in disease_has_symptons.items():
    i += 1
    if i==2:
        disease2 = value
        break
    else:
        disease1 = value
    

In [50]:
print(disease1[0])
node1 = onto.search_one(iri='*HP_0005978').iri
print(node1)

HP:0005978
http://purl.obolibrary.org/obo/HP_0005978


In [54]:
print(disease1)

['HP_0005978', 'HP_0001347', 'HP_0000035', 'HP_0000405', 'HP_0000842', 'HP_0000618', 'HP_0000518', 'HP_0000987', 'HP_0000563', 'HP_0000648', 'HP_0007675', 'HP_0000602', 'HP_0008046', 'HP_0000512', 'HP_0000407', 'HP_0000613', 'HP_0001513', 'HP_0001249', 'HP_0000431', 'HP_0000135', 'HP_0000639', 'HP_0008736', 'HP_0000501', 'HP_0000463', 'HP_0007703', 'HP_0005978', 'HP_0001347', 'HP_0000035', 'HP_0000405', 'HP_0000842', 'HP_0000618', 'HP_0000518', 'HP_0000987', 'HP_0000563', 'HP_0000648', 'HP_0007675', 'HP_0000602', 'HP_0008046', 'HP_0000512', 'HP_0000407', 'HP_0000613', 'HP_0001513', 'HP_0001249', 'HP_0000431', 'HP_0000135', 'HP_0000639', 'HP_0008736', 'HP_0000501', 'HP_0000463', 'HP_0007703', 'HP_0005978', 'HP_0001347', 'HP_0000035', 'HP_0000405', 'HP_0000842', 'HP_0000618', 'HP_0000518', 'HP_0000987', 'HP_0000563', 'HP_0000648', 'HP_0007675', 'HP_0000602', 'HP_0008046', 'HP_0000512', 'HP_0000407', 'HP_0000613', 'HP_0001513', 'HP_0001249', 'HP_0000431', 'HP_0000135', 'HP_0000639', 'HP_0

In [52]:
disease1 = [item.replace(':','_') for item in disease1]
disease2 = [item.replace(':','_') for item in disease2]
distance_matrix = np.zeros((len(disease1), len(disease2)), np.int8)
for i in range(len(disease1)):
    for j in range(len(disease2)):
        node1 = onto.search_one(iri='*%s'%disease1[i]).iri
        node2 = onto.search_one(iri='*%s'%disease2[j]).iri
        distance_matrix[i][j] = pairwise_distance_two_diseases(node1,node2,graph,q)[0][0][0]
        

KeyboardInterrupt: 

In [ ]:
print(distance_matrix)